In [ ]:
import xmltodict as xtd
import pandas as pd
import os.path
import pathlib
import datetime
from sqlalchemy import create_engine

In [ ]:
DIALECT = 'oracle'
SQL_DRIVER = 'cx_oracle'
USERNAME = 'YOUR_USERNAME'
PASSWORD = 'YOUR_PASSWORD'
HOST = 'YOUR_HOST'
PORT = 1540
SERVICE = 'YOUR_SERVICE'
ENGINE_PATH_WIN_AUTH = DIALECT + '+' + SQL_DRIVER + '://' + USERNAME + ':' + PASSWORD +'@' + HOST + ':' + str(PORT) + '/?service_name=' + SERVICE
engine = create_engine(ENGINE_PATH_WIN_AUTH, max_identifier_length=128)

In [ ]:
my_dir = r'E:\CognosReports'
count = 0
d = datetime.datetime.now()
data = []
data2 = []
data3 = []

In [ ]:
# SHEET 1 SINGLE QUERY
def sheet1_1(my_data):
    report_name = my_data['report']['reportName']
    query_name = my_data['report']['queries']['query']['@name']
    try:
        sql_query_name = my_data['report']['queries']['query']['source']['sqlQuery']['@name']
    except KeyError:
        sql_query_name = ''
    try:
        sql_query_text = my_data['report']['queries']['query']['source']['sqlQuery']['sqlText']
    except KeyError:
        sql_query_text = ''
    data.append([report_name, query_name, sql_query_name, sql_query_text])

In [ ]:
# SHEET 1 MULTIPLE QUERIES
def sheet1_multiple(query_count_file, my_data):
    report_name = my_data['report']['reportName']
    for ii in range(0, query_count_file):
        try:
            query_name = my_data['report']['queries']['query'][ii]['@name']
        except IndexError:
            query_name = ''
        except KeyError:
            query_name = ''
        try:
            sql_query_name = my_data['report']['queries']['query'][ii]['source']['sqlQuery']['@name']
            sql_query_text = my_data['report']['queries']['query'][ii]['source']['sqlQuery']['sqlText']
        except IndexError:
            sql_query_name = ''
            sql_query_text = ''
        except KeyError:
            sql_query_name = ''
            sql_query_text = ''
        data.append([report_name, query_name, sql_query_name, sql_query_text])

In [ ]:
# SHEET 2 SINGLE QUERY
def sheet2_1(my_data):
    report_name = my_data['report']['reportName']
    query_name = my_data['report']['queries']['query']['@name']
    di_list = '{}_di'.format(query_name)
    di_list = []
    die_list = '{}_die'.format(query_name)
    die_list = []
    di_total_count = (my_data['report']['queries']['query']['selection']['dataItem'])
    for d_item in di_total_count:
        if 'str' in str(type(d_item)):
            data_item_count = 1
            di_total = (my_data['report']['queries']['query']['selection']['dataItem']['@name'])
            die_total = (my_data['report']['queries']['query']['selection']['dataItem']['expression'])
            data2.append([report_name, query_name, di_total, die_total])
            break
        elif 'OrderedDict' in str(type(d_item)):
            di_total_count = len(my_data['report']['queries']['query']['selection']['dataItem'])
            try:
                for d_item_count in range(0, di_total_count):
                    di = my_data['report']['queries']['query']['selection']['dataItem'][d_item_count]['@name']
                    di_list.append(str(di).replace('[', '').replace(']', '').replace(')', '').replace('(', ''))
                    die = my_data['report']['queries']['query']['selection']['dataItem'][d_item_count]['expression']
                    die_list.append(str(die).replace('[', '').replace(']', '').replace(')', '').replace('(', ''))
                di_total = "".join(str(di_list))
                die_total = "".join(str(die_list))
            except KeyError:
                di_total = ''
        die_total = ''
        d_count = len(di_list)
        d_count2 = int(0)
        for x in di_list:
            if d_count2 == d_count:
                print("dcount & dcount2 = 0")
            elif d_count2 < d_count:
                die = die_list[d_count2]
                data2.append([report_name, query_name, x, die])
                d_count2 += 1

In [ ]:
# SHEET 2 MULTIPLE QUERIES
def sheet2_multiple(query_count_file, my_data):
    report_name = my_data['report']['reportName']
    for ii in range(0, query_count_file):
        query_name = my_data['report']['queries']['query'][ii]['@name']
        di_list = '{}_di'.format(query_name)
        di_list = []
        die_list = '{}_die'.format(query_name)
        die_list = []
        di_total_count = (my_data['report']['queries']['query'][ii]['selection']['dataItem'])
        for d_item in di_total_count:
            if 'str' in str(type(d_item)):
                di_total_count = 1
                di_total = (my_data['report']['queries']['query'][ii]['selection']['dataItem']['@name'])
                die_total = (my_data['report']['queries']['query'][ii]['selection']['dataItem']['expression'])
                data2.append([report_name, query_name, di_total, die_total])
                break
            elif 'OrderedDict' in str(type(d_item)):
                di_total_count = len(my_data['report']['queries']['query'][ii]['selection']['dataItem'])
                try:
                    for d in range(0, di_total_count):
                        di = my_data['report']['queries']['query'][ii]['selection']['dataItem'][d]['@name']
                        di_list.append(str(di).replace('[', '').replace(']', '').replace(')', '').replace('(', ''))
                        die = my_data['report']['queries']['query'][ii]['selection']['dataItem'][d]['expression']
                        die_list.append(str(die).replace('[', '').replace(']', '').replace(')', '').replace('(', ''))
                    di_total = "".join(str(di_list))
                    die_total = "".join(str(die_list))
                except KeyError:
                    di_total = ''
                    die_total = ''
            d_count = len(di_list)
            d_count2 = int(0)
            for x in di_list:
                if d_count2 == d_count:
                    pass
                elif d_count2 < d_count:
                    die = die_list[d_count2]
                    data2.append([report_name, query_name, x, die])
                    d_count2 += 1
            break

In [ ]:
# SHEET 3 SINGLE QUERY
def sheet3_1(my_data):
    report_name = my_data['report']['reportName']
    query_name = my_data['report']['queries']['query']['@name']
    fe_list = '{}_fi'.format(query_name)
    fe_list = []
    try:
        fe_total = (my_data['report']['queries']['query']['detailFilters']['detailFilter'])
        for f_expression in (fe_total):
            if 'str' in str(type(f_expression)):
                fe_count = 1
                fe_total = my_data['report']['queries']['query']['detailFilters']['detailFilter']['filterExpression']
                data3.append([report_name, query_name, fe_total])
                break
            elif 'OrderedDict' in str(type(f_expression)):
                fe_count = len(my_data['report']['queries']['query']['detailFilters']['detailFilter'])
                try:
                    for f in range(0, fe_count):
                        fe = my_data['report']['queries']['query']['detailFilters']['detailFilter'][f]['filterExpression']
                        fe_list.append(str(fe).replace('[', '').replace(']', '').replace('(', '').replace(')', ''))
                    fe_total = "".join(str(fe_list))
                except KeyError:
                    fe_total = ''
            f_count = int(len(fe_list))
            f_count2 = int(0)
            for f in fe_list:
                if f_count2 == f_count:
                    fe_count = 'NONE'
                    data3.append([report_name, query_name, fe_count])
                elif f_count2 < f_count:
                    data3.append([report_name, query_name, f])
            break
    except KeyError:
        fe_total = ''

In [ ]:
# SHEET 3 MULTIPLE QUERIES
def sheet3_multiple(query_count_file, my_data):
    report_name = my_data['report']['reportName']
    for ii in range(0, query_count_file):
        query_name = my_data['report']['queries']['query'][ii]['@name']
        fe_list = '{}_fi'.format(query_name)
        fe_list = []
        try:
            fe_count = (my_data['report']['queries']['query'][ii]['detailFilters']['detailFilter'])
            for f_expression in (fe_count):
                if 'str' in str(type(f_expression)):
                    fe_count = 1
                    fe_total = my_data['report']['queries']['query'][ii]['detailFilters']['detailFilter']['filterExpression']
                    data3.append([report_name, query_name, fe_total.replace('[', '').replace(']', '').replace('(', '').replace(')', '')])
                    break
                elif 'OrderedDict' in str(type(f_expression)):
                    fe_count = len(my_data['report']['queries']['query'][ii]['detailFilters']['detailFilter'])
                    try:
                        for filter_e in range(0, fe_count):
                            fe = my_data['report']['queries']['query'][ii]['detailFilters']['detailFilter'][filter_e]['filterExpression']
                            fe_list.append(str(fe).replace('[', '').replace(']', '').replace('(', '').replace(')', ''))
                        fe_total = "".join(str(fe_list))
                    except KeyError:
                        fe_total = ''
                    f_count = int(len(fe_list))
                    f_count2 = int(0)
                    for f in fe_list:
                        if f_count2 == f_count:
                            fe_count = 'NONE'
                            data3.append([report_name, query_name, fe_count])
                        elif f_count2 < f_count:
                            data3.append([report_name, query_name, f])
                            f_count2 += 1
                    break
        except KeyError:
            fe_count = ''

In [ ]:
# LOG
def log_entry1(entry):
    logDir = r'J:\UltiPro - ITReports\LogFile'
    historyFile = '{}.log'.format(logDir)
    with open(historyFile, 'a') as hf:
        hf.write('{}: UP_BI_REPORTS\n {}\n\n'.format(d.strftime('%m/%d/%Y %H:%M'), entry))
        hf.close()
        
def log_entry2(entry):
    logDir = r'J:\UltiPro - ITReports\LogFile'
    historyFile = '{}.log'.format(logDir)
    with open(historyFile, 'a') as hf:
        hf.write('{}: UP_BI_DATA_ITEMS\n {}\n\n'.format(d.strftime('%m/%d/%Y %H:%M'), entry))
        hf.close()

def log_entry3(entry):
    logDir = r'J:\UltiPro - ITReports\LogFile'
    historyFile = '{}.log'.format(logDir)
    with open(historyFile, 'a') as hf:
        hf.write('{}: UP_BI_FILTERS\n {}\n\n'.format(d.strftime('%m/%d/%Y %H:%M'), entry))
        hf.close()

In [ ]:
# GATHER DATA
def gather_data_table(file_name):
    with open(file_name, 'r') as tf:
        try:
            try:
                xml_contents = tf.read()
                my_data = xtd.parse(xml_contents)
            except xtd.expat.ExpatError:
                print("Expat Error")
            report_name = my_data['report']['reportName']
            engine.execute("DELETE FROM UP_BI_REPORTS_3 WHERE REPORT_NAME = :x", x=report_name)
            engine.execute("DELETE FROM UP_BI_DATA_ITEMS_3 WHERE REPORT_NAME = :x", x=report_name)
            engine.execute("DELETE FROM UP_BI_FILTERS_3 WHERE REPORT_NAME = :x", x=report_name)
            print("Report name: {}".format(report_name))
            query_count_list = my_data['report']['queries']['query']
            for x in query_count_list:
                if 'str' in str(type(x)):
                    query_count_file = 1
                    break
                if 'OrderedDict' in str(type(x)):
                    query_count_file = len(my_data['report']['queries']['query'])
                    break
            # print("Query count: {}".format(query_count_file))
            if query_count_file == 1:
                sheet1_1(my_data)
                sheet2_1(my_data)
                sheet3_1(my_data)
            elif query_count_file >= 2:
                sheet1_multiple(query_count_file, my_data)
                sheet2_multiple(query_count_file, my_data)
                sheet3_multiple(query_count_file, my_data)
        except UnboundLocalError:
            print("Unbound Local Error")

In [ ]:
for file_name in os.listdir(my_dir):
    if file_name.endswith(".xml"):
        print("Opening: {}".format(file_name))
        file_name = r'{}\{}'.format(my_dir, file_name)
        gather_data_table(file_name)
        count += 1

In [ ]:
# CREATE DATA FRAMES
df1 = pd.DataFrame(columns=['REPORT_NAME', 'QUERY_NAME', 'SQL_NAME', 'SQL_TEXT'], data=data)
df1.drop_duplicates(inplace=True)
df1a = df1.values
df2 = pd.DataFrame(columns=['REPORT_NAME', 'QUERY_NAME', 'DATA_ITEM_NAME', 'DATA_ITEM_EXPRESSION'], data=data2)
df2.drop_duplicates(inplace=True)
df2a = df2.values
df3 = pd.DataFrame(columns=['REPORT_NAME', 'QUERY_NAME', 'FILTER_EXPRESSION'], data=data3)
df3.drop_duplicates(inplace=False)
df3a = df3.values

In [ ]:
# LOG DATAFRAMES
log_entry1(df1.to_string(index=False))
log_entry2(df2.to_string(index=False))
log_entry3(df3.to_string(index=False))

In [ ]:
# WRITE TO EXCEL
writer = pd.ExcelWriter(r'E:\CognosReports\Excel Reports\Excel Report {}.xlsx'.format(d.strftime('%m-%d-%Y')),
                        engine='xlsxwriter')
df1.to_excel(writer, sheet_name='UP_BI_REPORTS', index=False)
df2.to_excel(writer, sheet_name='UP_BI_DATA_ITEMS', index=False)
df3.to_excel(writer, sheet_name='UP_BI_FILTERS', index=False)

len_1 = len(df1.index)
len_2 = len(df2.index)
len_3 = len(df3.index)

def format_excel(writer):
    workbook = writer.book
    worksheet1 = writer.sheets['UP_BI_REPORTS']
    worksheet2 = writer.sheets['UP_BI_DATA_ITEMS']
    worksheet3 = writer.sheets['UP_BI_FILTERS']
    worksheet1.set_column('A:A', 60)
    worksheet2.set_column('A:A', 60)
    worksheet3.set_column('A:A', 60)
    worksheet1.set_column('B:B', 40)
    worksheet2.set_column('B:B', 40)
    worksheet3.set_column('B:B', 40)
    worksheet1.set_column('C:C', 52)
    worksheet2.set_column('C:C', 52)
    worksheet3.set_column('C:C', 162.7)
    worksheet1.set_column('D:D', 110)
    worksheet2.set_column('D:D', 110)
    worksheet1.add_table(0, 0, len_1, 3, {'data': df1a,
                                          'style': 'Table Style Medium 2',
                                          'columns': [{'header': 'REPORT_NAME'},
                                                      {'header': 'QUERY_NAME'},
                                                      {'header': 'SQL_NAME'},
                                                      {'header': 'SQL_TEXT'},
                                                      ]})
    worksheet2.add_table(0, 0, len_2, 3, {'data': df2a,
                                          'style': 'Table Style Medium 2',
                                          'columns': [{'header': 'REPORT_NAME'},
                                                      {'header': 'QUERY_NAME'},
                                                      {'header': 'DATA_ITEM_NAME'},
                                                      {'header': 'DATA_ITEM_EXPRESSION'},
                                                      ]})
    worksheet3.add_table(0, 0, len_3, 2, {'data': df3a,
                                          'style': 'Table Style Medium 2',
                                          'columns': [{'header': 'REPORT_NAME'},
                                                      {'header': 'QUERY_NAME'},
                                                      {'header': 'FILTER_EXPRESSION'},
                                                      ]})
format_excel(writer)
writer.save()

In [ ]:
df1.to_sql('up_bi_reports_3', con=engine, if_exists='append', index=False)
df2.to_sql('up_bi_data_items_3', con=engine, if_exists='append', index=False)
df3.to_sql('up_bi_filters_3', con=engine, if_exists='append', index=False)